# 2. มี rest คั่นทุกตัว และ rest เท่ากันทั้งหมด

In [1]:
from midiutil import MIDIFile
from mingus.core import chords

import os
import csv
import subprocess
import random

In [2]:
NOTES = ["C", "C#", "D", "Eb", "E", "F", "F#", "G", "Ab", "A", "Bb", "B"]
OCTAVES = list(range(11))
NOTES_IN_OCTAVE = len(NOTES)

errors = {"notes": "Bad input, please refer this spec-\n"}

In [3]:
def swap_accidentals(note):
    if note == "Db": 
        return "C#"
    if note == "D#":
        return "Eb"
    if note == "E#":
        return "F"
    if note == "Gb":
        return "F#"
    if note == "G#":
        return "Ab"
    if note == "A#":
        return "Bb"
    if note == "B#":
        return "C"


In [4]:
def note_to_number(note: str, octave: int) -> int:
    note = swap_accidentals(note)
    assert note in NOTES, errors["notes"]
    assert octave in OCTAVES, errors["notes"]

    note = NOTES.index(note)
    note += NOTES_IN_OCTAVE * octave

    assert 0 <= note <= 127, errors["notes"]

    return note


# fluidsynth_executable, '-i', C:\Work2_2566\Project2\Top 18 Free Piano Soundfonts\KAWAI good piano.sf2,      '-F', C:\Work2_2566\Project2\data_real\norest_1\output.wav, C:\Work2_2566\Project2\data_real\norest_1\midi_000.mid
def fs(input_path, output_path):
    fluidsynth_executable = "C:\\Users\\user\\fluidsynth-install\\bin\\fluidsynth.exe"
    soundfont_path = (
        "C:\\Work1_2567\\Project\\Top_18_Free_Piano_Soundfonts\\Porter_Grand_Piano.sf2"
    )
    midi_file = (
        input_path
    )
    output_file = (
        output_path
    )

    # Construct the command
    command = [
        fluidsynth_executable,
        "-i",
        soundfont_path,
        "-F",
        output_file,
        midi_file,
    ]

    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")

    # delete midi file : keep only audio file
    os.remove(midi_file)

In [5]:
array_of_notes = NOTES  # ตั้งว่าจะเอาโน๊ตอะไรบ้าง

array_of_note_numbers = []
array_temp = []
for note in range(len(array_of_notes)):
    array_temp = []
    OCTAVE = 4  # เปลี่ยน octive ตรงนี้
    array_of_note_numbers.append(note)
print(array_of_note_numbers)

track = 0
channel = 0
time = 0  # In beats
duration = 2  # In beats ตัวขาว
tempo = 120  # In BPM
volume = 255  # 0-127, as per the MIDI standard

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


# 2.1.1 ลำดับของ pitch แตกต่างกัน

In [6]:
def create_files_2_1_1(
    array_of_note_numbers,
    current_depth,
    max_depth,
    path,
    writer,
    rhythm,
    rest,
    notes_prefix="",
    indices_prefix="",
    track=0,
    channel=0,
    tempo=120,
    time=0, 
    duration=2,
    duration_rest=2,
    volume=255,
):
    if current_depth == max_depth:
        # print(indices_prefix)
        MyMIDI = MIDIFile(1)
        MyMIDI.addTempo(track, time, tempo)
        note_indices = [int(x) for x in indices_prefix.split()]
        note_indices_strip = "".join(indices_prefix.split())
        label_indice = ",".join(indices_prefix.split())

        for idx in range(max_depth):
            MyMIDI.addNote(
                track, channel, note_indices[idx] + 48, time, duration, volume
            )
            time += duration
            time += duration_rest

        midi_file_name = f"{path}/midi_{notes_prefix+note_indices_strip}.mid"
        with open(midi_file_name, "wb") as output_file:
            MyMIDI.writeFile(output_file)
        fs(f'{path}/midi_{notes_prefix+note_indices_strip}.mid', f'{path}/wav_{notes_prefix+note_indices_strip}.wav')
        writer.writerow([f"wav_{notes_prefix+note_indices_strip}.wav", f"[{label_indice}]", tempo, rhythm, rest])
    else:
        for i, pitch in enumerate(array_of_note_numbers):
            # print(indices_prefix)
            create_files_2_1_1(
                array_of_note_numbers,
                current_depth + 1,
                max_depth,
                path,
                writer,
                rhythm,
                rest,
                notes_prefix + NOTES[i],
                indices_prefix + f"{i} ",
            )

In [7]:
# โน้ต 3 ตัว
rhythm = "all white"
rest = "all half rest"
# ระบุเส้นทางไปยังโฟลเดอร์ที่มีไฟล์ที่คุณต้องการเก็บข้อมูล
folder_path = "../../data/exp2-1/2-1-1"
# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
csv_file = "../../data/exp2-1/2-1-1/file_names_exp2-1-1.csv"

with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File Name', 'label', 'tempo', 'rhythm', "rest"])

    path = "../../data/exp2-1/2-1-1"
    max_depth = 3  # we have three levels of loops in the original code
    create_files_2_1_1(array_of_note_numbers, 0, max_depth, path, writer, rhythm, rest)

print(f"File names have been written to {csv_file}")

File names have been written to ../../data/exp2-1/2-1-1/file_names_exp2-1-1.csv


# 2.1.2 ลำดับของ pitch, tempo แตกต่างกัน

In [8]:
def create_files_2_1_2(
    array_of_note_numbers,
    current_depth,
    max_depth,
    path,
    writer,
    rhythm,
    rest,
    notes_prefix="",
    indices_prefix="",
    track=0,
    channel=0,
    tempo=120,
    time=0, 
    duration=2,
    duration_rest=2,
    volume=255,
):
    if current_depth == max_depth:
        # print(indices_prefix)
        MyMIDI = MIDIFile(1)
        tempo = random.choice([60, 80, 100, 120, 140, 160, 180])
        MyMIDI.addTempo(track, time, tempo)
        note_indices = [int(x) for x in indices_prefix.split()]
        note_indices_strip = "".join(indices_prefix.split())
        label_indice = ",".join(indices_prefix.split())

        for idx in range(max_depth):
            MyMIDI.addNote(
                track, channel, note_indices[idx] + 48, time, duration, volume
            )
            time += duration
            time += duration_rest

        midi_file_name = f"{path}/midi_{notes_prefix+note_indices_strip}.mid"
        with open(midi_file_name, "wb") as output_file:
            MyMIDI.writeFile(output_file)
        fs(f'{path}/midi_{notes_prefix+note_indices_strip}.mid', f'{path}/wav_{notes_prefix+note_indices_strip}.wav')
        writer.writerow([f"wav_{notes_prefix+note_indices_strip}.wav", f"[{label_indice}]", tempo, rhythm, rest])
    else:
        for i, pitch in enumerate(array_of_note_numbers):
            # print(indices_prefix)
            create_files_2_1_2(
                array_of_note_numbers,
                current_depth + 1,
                max_depth,
                path,
                writer,
                rhythm,
                rest,
                notes_prefix + NOTES[i],
                indices_prefix + f"{i} ",
            )

In [9]:
# โน้ต 3 ตัว
rhythm = "all white"
rest = "all half rest"
# ระบุเส้นทางไปยังโฟลเดอร์ที่มีไฟล์ที่คุณต้องการเก็บข้อมูล
folder_path = "../../data/exp2-1/2-1-2"
# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
csv_file = "../../data/exp2-1/2-1-2/file_names_exp2-1-2.csv"

with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File Name', 'label', 'tempo', 'rhythm', "rest"])

    path = "../../data/exp2-1/2-1-2"
    max_depth = 3  # we have three levels of loops in the original code
    create_files_2_1_2(array_of_note_numbers, 0, max_depth, path, writer, rhythm, rest)

print(f"File names have been written to {csv_file}")

File names have been written to ../../data/exp2-1/2-1-2/file_names_exp2-1-2.csv


# 2.1.3 ลำดับของ pitch, rhythm แตกต่างกัน

In [10]:
def create_files_2_1_3(
    array_of_note_numbers,
    current_depth,
    max_depth,
    path,
    writer,
    rhythm,
    rest,
    notes_prefix="",
    indices_prefix="",
    track=0,
    channel=0,
    tempo=120,
    time=0, 
    duration=2,
    duration_rest=2,
    volume=255,
):
    if current_depth == max_depth:
        # print(indices_prefix)
        MyMIDI = MIDIFile(1)
        # tempo = random.choice([60, 80, 100, 120, 140, 160, 180])
        rhythm = {4:"whole", 2:"white", 1:"black", 0.5:"half", 0.25:"Quater"}
        MyMIDI.addTempo(track, time, tempo)
        note_indices = [int(x) for x in indices_prefix.split()]
        note_indices_strip = "".join(indices_prefix.split())
        label_indice = ",".join(indices_prefix.split())
        all_rhythm = []
        for idx in range(max_depth):
            duration = random.choice([4, 2, 1, 0.5, 0.25])

            MyMIDI.addNote(
                track, channel, note_indices[idx] + 48, time, duration, volume
            )
            time += duration
            all_rhythm.append(rhythm[duration])
            time += duration_rest

        midi_file_name = f"{path}/midi_{notes_prefix+note_indices_strip}.mid"
        with open(midi_file_name, "wb") as output_file:
            MyMIDI.writeFile(output_file)
        fs(f'{path}/midi_{notes_prefix+note_indices_strip}.mid', f'{path}/wav_{notes_prefix+note_indices_strip}.wav')
        writer.writerow([f"wav_{notes_prefix+note_indices_strip}.wav", f"[{label_indice}]", tempo, all_rhythm, rest])
    else:
        for i, pitch in enumerate(array_of_note_numbers):
            # print(indices_prefix)
            create_files_2_1_3(
                array_of_note_numbers,
                current_depth + 1,
                max_depth,
                path,
                writer,
                rhythm,
                rest,
                notes_prefix + NOTES[i],
                indices_prefix + f"{i} ",
            )

In [11]:
# โน้ต 3 ตัว
rhythm = "all white"
rest = "all half rest"
# ระบุเส้นทางไปยังโฟลเดอร์ที่มีไฟล์ที่คุณต้องการเก็บข้อมูล
folder_path = "../../data/exp2-1/2-1-3"
# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
csv_file = "../../data/exp2-1/2-1-3/file_names_exp2-1-3.csv"

with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File Name', 'label', 'tempo', 'rhythm', "rest"])

    path = "../../data/exp2-1/2-1-3"
    max_depth = 3  # we have three levels of loops in the original code
    create_files_2_1_3(array_of_note_numbers, 0, max_depth, path, writer, rhythm, rest)

print(f"File names have been written to {csv_file}")

File names have been written to ../../data/exp2-1/2-1-3/file_names_exp2-1-3.csv


# 1.1.4 ลำดับของ pitch, rhythm, tempo แตกต่างกัน

In [12]:
def create_files_2_1_4(
    array_of_note_numbers,
    current_depth,
    max_depth,
    path,
    writer,
    rhythm,
    rest,
    notes_prefix="",
    indices_prefix="",
    track=0,
    channel=0,
    tempo=120,
    time=0, 
    duration=2,
    duration_rest=2,
    volume=255,
):
    if current_depth == max_depth:
        # print(indices_prefix)
        MyMIDI = MIDIFile(1)
        tempo = random.choice([60, 80, 100, 120, 140, 160, 180])
        rhythm = {4:"whole", 2:"white", 1:"black", 0.5:"half", 0.25:"Quater"}
        MyMIDI.addTempo(track, time, tempo)
        note_indices = [int(x) for x in indices_prefix.split()]
        note_indices_strip = "".join(indices_prefix.split())
        label_indice = ",".join(indices_prefix.split())

        all_rhythm = []
        for idx in range(max_depth):
            duration = random.choice([4, 2, 1, 0.5, 0.25])

            MyMIDI.addNote(
                track, channel, note_indices[idx] + 48, time, duration, volume
            )
            time += duration
            all_rhythm.append(rhythm[duration])
            time += duration_rest

        midi_file_name = f"{path}/midi_{notes_prefix+note_indices_strip}.mid"
        with open(midi_file_name, "wb") as output_file:
            MyMIDI.writeFile(output_file)
        fs(f'{path}/midi_{notes_prefix+note_indices_strip}.mid', f'{path}/wav_{notes_prefix+note_indices_strip}.wav')
        writer.writerow([f"wav_{notes_prefix+note_indices_strip}.wav", f"[{label_indice}]", tempo, all_rhythm, rest])
    else:
        for i, pitch in enumerate(array_of_note_numbers):
            # print(indices_prefix)
            create_files_2_1_4(
                array_of_note_numbers,
                current_depth + 1,
                max_depth,
                path,
                writer,
                rhythm,
                rest,
                notes_prefix + NOTES[i],
                indices_prefix + f"{i} ",
            )

In [13]:
# โน้ต 3 ตัว
rhythm = "all white"
rest = "all half rest"
# ระบุเส้นทางไปยังโฟลเดอร์ที่มีไฟล์ที่คุณต้องการเก็บข้อมูล
folder_path = "../../data/exp2-1/2-1-4"
# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
csv_file = "../../data/exp2-1/2-1-4/file_names_exp2-1-4.csv"

with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File Name', 'label', 'tempo', 'rhythm', "rest"])

    path = "../../data/exp2-1/2-1-4"
    max_depth = 3  # we have three levels of loops in the original code
    create_files_2_1_4(array_of_note_numbers, 0, max_depth, path, writer, rhythm, rest)

print(f"File names have been written to {csv_file}")

File names have been written to ../../data/exp2-1/2-1-4/file_names_exp2-1-4.csv
